In [1]:

import string
import matplotlib.pyplot as plt
import nltk
import sys
import os
import regex as re
import itertools
# from nltk.corpus import stopwords
newdir = os.path.abspath(os.path.join(os.path.dirname("__file__"), './src'))
print(newdir)
sys.path.append(newdir)
# nltk.download("stopwords")


/Users/gauravrai/Desktop/SSC/WAT/src


In [25]:
rgx = re.compile("([\w][\w']*\w)")
s = "John's mom went there, but he wasn't there. So she said: 'Where are you'"
words = rgx.findall(s)
count = 3

In [3]:
lists = [words[i:i+count] for i in range(0, len(words), count)]

In [4]:
lists

[["John's", 'mom', 'went'],
 ['there', 'but', 'he'],
 ["wasn't", 'there', 'So'],
 ['she', 'said', 'Where'],
 ['are', 'you']]

In [5]:
res = list(map(list, zip(words, words[1:], words[2:])))

In [6]:
res

[["John's", 'mom', 'went'],
 ['mom', 'went', 'there'],
 ['went', 'there', 'but'],
 ['there', 'but', 'he'],
 ['but', 'he', "wasn't"],
 ['he', "wasn't", 'there'],
 ["wasn't", 'there', 'So'],
 ['there', 'So', 'she'],
 ['So', 'she', 'said'],
 ['she', 'said', 'Where'],
 ['said', 'Where', 'are'],
 ['Where', 'are', 'you']]

In [7]:
def check(sentence, words):
    res = list(map(lambda x: all(map(lambda y:y in x.split(),
                                          words)), sentence))
    return [sentence[i] for i in range(0, len(res)) if res[i]]

In [13]:
sentence = ["John wasn't present when John's mother arrived",'She then cried out',"Where are you!"]
sd = "John wasn\'t present when John's mother arrived. She then cried out, \"Where are you!\" "
sd2 = "John wasn\'t present when John's mother arrived. She then cried out, \"Where are you!\" "

[]

In [31]:
sd = sd.strip(string.punctuation)
newres = []
for r in res:
    newres.append(r[0]+" "+r[1]+" "+r[2])
pattern = r'.*\b(?='+'|'.join(newres) + r')\b.*'    
matches = re.findall(pattern, sd, re.IGNORECASE)
matches

['John wasn\'t present when John\'s mother arrived. She then cried out, "Where are you!" ']

In [ ]:
def direct_phrase(given_paragraph, user_paragraph):

    triple_wise_list = []
    
    # Looping through each sentence in the given paragraph
    for sentence in given_paragraph:
        words  = split_into_words(sentence)
        # Generating triplets from the consecutive words in the sentence
        sentence_triple_list = list(map(list, zip(words, words[1:], words[2:])))
        triple_wise_list.append(sentence_triple_list)

    # Appending inner elements to the triplet list for feeding into regex function
    triple_list = []
    for words in triple_wise_list:
        triple_list.append(words[0]+" "+words[1]+" "+words[2])

    # Removing punctuations from the user paragraph
    user_paragraph = user_paragraph.strip(string.punctuation)

    # Regex operation to the direct phrase
    pattern = r'.*\b(?='+'|'.join(triple_list) + r')\b.*'
    matches = re.findall(pattern, user_paragraph, re.IGNORECASE)
    return matches


# Funtion to split the sentence into words using Regex
def split_into_words(text):
    rgx = re.compile("([\w][\w']*\w)")
    words = rgx.findall(text)
    return words
        

In [11]:
import src
from src.paraphraser.parrot import Parrot
from src.models import Model
model = Model()
parrot = Parrot(model)

In [12]:
#grammar corrector
from happytransformer import TTSettings
from happytransformer import HappyTextToText
happy_tt = HappyTextToText("T5", "vennify/t5-base-grammar-correction")
beam_settings =  TTSettings(num_beams=5, min_length=1, max_length=100)

08/04/2022 09:44:37 - INFO - happytransformer.happy_transformer -   Using model: cpu


In [13]:
sentences = [
    "This query has taken my invaluable time in the morning",
    "The quick brown fox jumps over the lazy dog",
    "He taught us how to catch errors and how not to write",
    "He walked in crumbling tennis shoes and matched awkwardly like people used to in the seventies",
    "The colors used in the comforter are loud and bright"
]

In [55]:
def correct_grammar(sen):
    correct_sen = happy_tt.generate_text(sen, args=beam_settings).text
    return correct_sen

In [60]:
for i, text in enumerate(sentences):
    para_phrases = parrot.augment(input_phrase=text,
                                  use_gpu=False,
                                  diversity_ranker="levenshtein",
                                  do_diverse=False,
                                  max_return_phrases=10,
                                  max_length=32,
                                  adequacy_threshold=0.23,
                                  fluency_threshold=0.75)
    para_phrases = [[correct_grammar(para_phrase[0]), para_phrase[1]]
                    for para_phrase in para_phrases]
    print(f"{i + 1}:", para_phrases)


1: [['This question took my time in the morning.', 33], ['This question took up my precious time in the morning.', 33], ['This question took my precious time in the morning.', 32], ['This question took my valuable time in the morning.', 24], ['This question has taken my valuable time in the morning.', 19]]
2: [['The swift brown fox jumps over the lazy dog.', 16], ['The quick brown fox leaps over the lazy dog.', 15], ['The quick brown fox jumps over the lazy dog.', 12]]
3: [['He taught us to catch mistakes and how not to write.', 23], ['He taught us how to detect mistakes and how not to write.', 23], ['He taught us how to catch mistakes and not to write.', 23], ['He taught us how to catch errors and not write.', 19], ['He taught us how to catch errors and not to write.', 16], ['He taught us to catch errors and how not to write.', 16], ['He taught us how to catch errors and how not to write.', 12]]
4: [["He walked in sloppy tennis shoes and met awkwardly the way people wore in the '70s."